In [1]:
import numpy as np
import csv
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score, \
    recall_score, confusion_matrix, classification_report, \
    accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import StratifiedKFold

/home/lucia/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In this script, we have three ML models:

SVC: C-Support Vector Classification.
SGDClassifier: Linear classifiers (SVM, logistic regression, a.o.) with SGD training.
This estimator implements regularized linear models with stochastic gradient descent (SGD) learning: the gradient of the loss is estimated each sample at a time and the model is updated along the way with a decreasing strength schedule (aka learning rate). SGD allows minibatch (online/out-of-core) learning, see the partial_fit method. For best results using the default learning rate schedule, the data should have zero mean and unit variance.

Random Forest:
RF gives the best result:
[0.92454523 0.97484841 0.91408451 0.95157967 0.94434389]
f1: 0.94 (+/- 0.04) (positive class) 
precision: 0.9166666666666667
recall: 0.7476190476190475

Rule based: F1: 0.98245614   Recall: 1.0 Precision: 0.8125

The ML classifers are conduction in the following sequence:
1. grid search for best parameters
   pipline: standardscale, classifer(stratified 5 fold)
2. run best parameters again on the 5 fold model and display f1, precision and recall 

The random forest model shows the best result.
The best parameters are run on the full data with train/test set, of which the model is stored as the BestModel object



In [2]:
class myQuote:
    def __init__(self, text):
        #read in CSV, process
        self.quoteText = text
        self.quoteID = hash(self.quoteText)
        self.link = []
        self.name = []
        self.description = []
        self.scores = []
        self.cos = []
    
    def __hash__(self):
        return self.quoteID
    
    def __str__(self):
        return "Object text: " + self.quoteText + '\n' +\
        "Links: " + str(self.link) + '\n' +\
        "Names: " + str(self.name) + '\n' +\
        "Description " + str(self.description) + '\n' +\
        "Scores: " + str(self.scores)
        

In [3]:
infile = open('searchResults','rb')
results = pickle.load(infile, encoding='bytes')

In [4]:
labels = pd.read_csv('./LableResults/MLData_recode_mannual.csv')
label = labels['label'].values
label = label.astype(np.float)
print(len(label))
print(type(label))


200
<class 'numpy.ndarray'>


In [16]:
#get the first 200 labels: those are the one in the mannual annotate sample
def append_features(ob):
    count = 0
    proto_matrix = []
    for item in ob:
        col2 = np.append(ob[item].scores, ob[item].cos)
        #print(col2)
        proto_matrix.append(col2)
        count += 1
        if count > 199:
            break
            
            
    return proto_matrix
            
proto_matrix = append_features(results)
FeatureMatrix = np.matrix(proto_matrix)

In [18]:
FeatureMatrix[0:10][0:3]

matrix([[1.        , 1.        , 5.        , 7.        , 5.        ,
         7.        , 1.        , 1.        , 6.        , 0.        ,
         0.90142585],
        [5.        , 2.        , 3.        , 1.        , 1.        ,
         1.        , 1.        , 1.        , 3.        , 0.        ,
         0.97699932],
        [5.        , 7.        , 3.        , 5.        , 9.        ,
         9.        , 5.        , 1.        , 1.        , 0.        ,
         0.96864645]])

In [23]:
#data
X = FeatureMatrix
y = label

In [42]:
#Normalize data with standardscaler
scaler = StandardScaler()

In [77]:
#####grid search (the parameters predict everything to one class, we should use a separated 
#sample for tuning parameters, but not enough cases so far)
cv = StratifiedKFold(n_splits=5)

# _macro_f1 = np.empty(shape=[4,3,11])
# for _k_i, _kernel in enumerate(['linear', 'poly', 'rbf', 'sigmoid']):
#     for _g_i, _gamma in enumerate([0.01, 0.001, 0.0001]):
#         for _c_i, _C in enumerate([0.1, 0.3, 0.5, 0.7, 0.9, 1.0, 1.3, 1.5, 1.7, 2.0, 3.0]):
#             _marco_1config = np.zeros(5)
#             for _f_i, (_train_idx, _test_idx) in enumerate(cv.split(X_train, y_train)):
#                 _x_t = X_tramake_pipeline(StandardScaler()in[_train_idx]
#                 _x_v = X_train[_test_idx]
#                 _y_t = y_train[_train_idx]
#                 _y_v = y_train[_test_idx]
#                 _scaler = StandardScaler()
#                 _scaler.fit(_x_t)
#                 _x_t_sc = _scaler.transform(_x_t)
#                 _x_v_sc = _scaler.transform(_x_v)
#                 _svc = svm.SVC(kernel=_kernel, gamma=_gamma, C=_C)
#                 _svc.fit(_x_t_sc, _y_t)
#                 _marco_1config[_f_i] = _svc.score(_x_v_sc, _y_v)
#             _macro_f1[_k_i, _g_i, _c_i] = np.mean(_marco_1config)
                
svc = make_pipeline(StandardScaler(),svm.SVC())
parameters = [{'svc__kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'svc__gamma': [0.01, 0.001, 0.0001],
                     'svc__C':[0.1, 0.3, 0.5, 0.7, 0.9, 1.0, 1.3, 1.5, 1.7, 2.0, 3.0] , 'svc__class_weight':['balanced']}]
                   
grid_search_item = GridSearchCV(estimator = svc,
                          param_grid = parameters,
                           cv =  cv,
                           scoring = 'accuracy',
                           n_jobs = -1)
grid_search = grid_search_item.fit(X, y)

print(grid_search.best_score_)####not sure how come this is so good,but the parameters don't work good in the model
print(grid_search.best_params_)

means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
params = grid_search.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

0.925
{'svc__C': 0.7, 'svc__class_weight': 'balanced', 'svc__gamma': 0.01, 'svc__kernel': 'linear'}
0.885000 (0.045127) with: {'svc__C': 0.1, 'svc__class_weight': 'balanced', 'svc__gamma': 0.01, 'svc__kernel': 'linear'}
0.850000 (0.014887) with: {'svc__C': 0.1, 'svc__class_weight': 'balanced', 'svc__gamma': 0.01, 'svc__kernel': 'poly'}
0.880000 (0.055596) with: {'svc__C': 0.1, 'svc__class_weight': 'balanced', 'svc__gamma': 0.01, 'svc__kernel': 'rbf'}
0.875000 (0.039481) with: {'svc__C': 0.1, 'svc__class_weight': 'balanced', 'svc__gamma': 0.01, 'svc__kernel': 'sigmoid'}
0.885000 (0.045127) with: {'svc__C': 0.1, 'svc__class_weight': 'balanced', 'svc__gamma': 0.001, 'svc__kernel': 'linear'}
0.430000 (0.332839) with: {'svc__C': 0.1, 'svc__class_weight': 'balanced', 'svc__gamma': 0.001, 'svc__kernel': 'poly'}
0.430000 (0.332839) with: {'svc__C': 0.1, 'svc__class_weight': 'balanced', 'svc__gamma': 0.001, 'svc__kernel': 'rbf'}
0.430000 (0.332839) with: {'svc__C': 0.1, 'svc__class_weight': 'ba

In [78]:
svc = make_pipeline(StandardScaler(),svm.SVC(gamma=0.001, class_weight='balanced', C = 0.7, kernel = 'linear'))
scores = cross_val_score(svc, X, y, cv=cv, scoring='f1_weighted')
print(scores)
print("f1: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
precision = cross_val_score(svc, X, y, cv=cv, scoring='precision')
recall = cross_val_score(svc, X, y, cv=cv, scoring='recall')
print(precision.mean())
print(recall.mean())

[0.88599492 0.95121951 0.92732491 0.92893773 0.95157967]
f1: 0.93 (+/- 0.05)
0.7176190476190476
0.9095238095238095


####SGDClassifier 

In [98]:
#####grid search (the parameters predict everything to one class, we should use a separated 
#sample for tuning parameters, but not enough cases so far)
clf = make_pipeline(StandardScaler(),SGDClassifier(max_iter= 1000))
#clf = SGDClassifier(max_iter= 1000)
parameters = [{'sgdclassifier__alpha': [0.01, 0.05, 0.001, 0.005], 'sgdclassifier__class_weight':['balanced'],
              'sgdclassifier__loss': ['hinge','log','modified_huber','squared_hinge', 'perceptron'], 
               'sgdclassifier__penalty':['none','l1','l2']}]
                   
grid_search_item = GridSearchCV(clf,
                          param_grid = parameters,
                           scoring = 'accuracy',
                           cv = cv,
                           n_jobs = -1)
grid_search = grid_search_item.fit(X, y)

print(grid_search.best_score_)####not sure how come this is so good,but the parameters don't work good in the model
print(grid_search.best_params_)

0.935
{'sgdclassifier__alpha': 0.05, 'sgdclassifier__class_weight': 'balanced', 'sgdclassifier__loss': 'squared_hinge', 'sgdclassifier__penalty': 'l2'}


In [85]:
clf.get_params().keys()

dict_keys(['memory', 'steps', 'standardscaler', 'sgdclassifier', 'standardscaler__copy', 'standardscaler__with_mean', 'standardscaler__with_std', 'sgdclassifier__alpha', 'sgdclassifier__average', 'sgdclassifier__class_weight', 'sgdclassifier__epsilon', 'sgdclassifier__eta0', 'sgdclassifier__fit_intercept', 'sgdclassifier__l1_ratio', 'sgdclassifier__learning_rate', 'sgdclassifier__loss', 'sgdclassifier__max_iter', 'sgdclassifier__n_iter', 'sgdclassifier__n_jobs', 'sgdclassifier__penalty', 'sgdclassifier__power_t', 'sgdclassifier__random_state', 'sgdclassifier__shuffle', 'sgdclassifier__tol', 'sgdclassifier__verbose', 'sgdclassifier__warm_start'])

In [99]:
clf = make_pipeline(StandardScaler(),SGDClassifier(max_iter=1000, alpha=0.01, class_weight = 'balanced', loss = 'squared_hinge', penalty= 'l2'))
scores = cross_val_score(clf, X, y, cv=cv, scoring='f1_weighted')
print(scores)
print("f1: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
precision = cross_val_score(clf, X, y, cv=cv, scoring='precision')
recall = cross_val_score(clf, X, y, cv=cv, scoring='recall')
print(precision.mean())
print(recall.mean())

[0.92454523 0.91023035 0.89142857 0.90315934 0.94871795]
f1: 0.92 (+/- 0.04)
0.7356060606060606
0.7761904761904763


###random forest

In [104]:
#####grid search (the parameters predict everything to one class, we should use a separated 
#sample for tuning parameters, but not enough cases so far)
rf = make_pipeline(StandardScaler(),RandomForestClassifier())

parameters = [{'randomforestclassifier__max_features':['auto','sqrt','log2'], 'randomforestclassifier__class_weight':['balanced'], 
               'randomforestclassifier__max_leaf_nodes':[10,50,100], 'randomforestclassifier__max_depth':[2,5,10,20], 'randomforestclassifier__n_estimators' : [50,100,200,300,400]}]
                   
grid_search_item = GridSearchCV(rf,
                          param_grid = parameters,
                           cv = cv,
                           scoring = 'accuracy',
                           n_jobs = -1)
grid_search = grid_search_item.fit(X_train, y_train)

print(grid_search.best_score_)####not sure how come this is so good,but the parameters don't work good in the model
print(grid_search.best_params_)

means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
params = grid_search.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

0.9428571428571428
{'randomforestclassifier__class_weight': 'balanced', 'randomforestclassifier__max_depth': 5, 'randomforestclassifier__max_features': 'sqrt', 'randomforestclassifier__max_leaf_nodes': 100, 'randomforestclassifier__n_estimators': 100}
0.928571 (0.038793) with: {'randomforestclassifier__class_weight': 'balanced', 'randomforestclassifier__max_depth': 2, 'randomforestclassifier__max_features': 'auto', 'randomforestclassifier__max_leaf_nodes': 10, 'randomforestclassifier__n_estimators': 50}
0.914286 (0.048179) with: {'randomforestclassifier__class_weight': 'balanced', 'randomforestclassifier__max_depth': 2, 'randomforestclassifier__max_features': 'auto', 'randomforestclassifier__max_leaf_nodes': 10, 'randomforestclassifier__n_estimators': 100}
0.928571 (0.030833) with: {'randomforestclassifier__class_weight': 'balanced', 'randomforestclassifier__max_depth': 2, 'randomforestclassifier__max_features': 'auto', 'randomforestclassifier__max_leaf_nodes': 10, 'randomforestclassif

In [107]:
rf = make_pipeline(StandardScaler(), RandomForestClassifier(n_estimators=100, max_depth=5, class_weight = 'balanced',
                             random_state=50, max_leaf_nodes = 100, max_features = 'sqrt'))
scores = cross_val_score(rf, X, y, cv=cv, scoring='f1_weighted')
print(scores)
print("f1: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
precision = cross_val_score(rf, X, y, cv=cv, scoring='precision')
recall = cross_val_score(rf, X, y, cv=cv, scoring='recall')
print(precision.mean())
print(recall.mean())

[0.92454523 0.97484841 0.91408451 0.95157967 0.94434389]
f1: 0.94 (+/- 0.04)
0.9166666666666667
0.7476190476190475


In [ ]:
## use the best parameters to make prediction 

In [136]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=30)

In [137]:
### Configure transformation and apply to training set
X_train = scaler.fit_transform(X_train)
rf_best = RandomForestClassifier(n_estimators=100, max_depth=5, class_weight = 'balanced',
                             random_state=50, max_leaf_nodes = 100, max_features = 'sqrt')
bestModel = rf_best.fit(X_train, y_train) 
### Perform transformation on testing set
X_test = scaler.transform(X_test)
y_pred = bestModel.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[47,  0],
       [ 3, 10]])

In [138]:
#F1 score of the positive class in binary classification or weighted average of the F1 scores of each class for the multiclass task.
f1_score(y_test, y_pred, average=None)

array([0.96907216, 0.86956522])

In [140]:
# dump object to pickles
filename = 'BestModel'
outfile = open(filename,'wb')

pickle.dump(bestModel,outfile)
outfile.close()

In [ ]:
####predict all data 

In [141]:
#load feature object 
class myQuote:
    def __init__(self, text):
        #read in CSV, process
        self.quoteText = text
        self.quoteID = hash(self.quoteText)
        self.link = []
        self.name = []
        self.description = []
        self.scores = []
        self.cos = []
    
    def __hash__(self):
        return self.quoteID
    
    def __str__(self):
        return "Object text: " + self.quoteText + '\n' +\
        "Links: " + str(self.link) + '\n' +\
        "Names: " + str(self.name) + '\n' +\
        "Description " + str(self.description) + '\n' +\
        "Scores: " + str(self.scores)

infile2 = open('searchResults','rb')
results = pickle.load(infile2, encoding='bytes')

In [142]:
#get the first 200 labels: those are the one in the mannual annotate sample
def append_features(ob):
    proto_matrix = []
    for item in ob:
        col2 = np.append(ob[item].scores, ob[item].cos)
        #print(col2)
        proto_matrix.append(col2)
      
    return proto_matrix
 
            
proto_matrix = append_features(results)
FeatureMatrix2 = np.matrix(proto_matrix)

In [145]:
scaled_matrix = scaler.transform(FeatureMatrix2)


In [149]:
y = rf_best.predict(scaled_matrix)
len(y)

4520

In [150]:
def Save_Results(objects,pre_results,fname,fname2):
    with open(fname , 'w') as f:
        writer = csv.writer(f, delimiter = ',',quoting=csv.QUOTE_MINIMAL)  
        writer.writerow(["hash"] + ["text"] + ["score"] + ["cosineSim"])
        
        for item in objects:
            writer.writerow([objects[item].quoteID] + [objects[item].quoteText] + [objects[item].scores] + [objects[item].cos])  
    f.close()
    
    with open(fname2 , 'w') as f:
        writer = csv.writer(f, delimiter = ',',quoting=csv.QUOTE_MINIMAL)  
        writer.writerow(["result"])
        
        for item in pre_results:
            writer.writerow([item])  
    f.close()
    
Save_Results(results, y, 'text_all2.csv', 'results_all2.csv')

#Rule based classifier

In [332]:
rule = pd.read_csv('./LableResults/EvaluateSample3_fscore.csv')
rule[0:5]

,hash,text,count,cosineSim,label,manual
0,-4.240000e+18,VIRUS SPREADING LIKE WILDFIRE ON FB. ITS A TRO...,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",0.956189,0,1
1,7.570000e+18,They say true friends go long periods of time ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",0.924270,0,1
2,3.580000e+18,Going to church doesn't make you a holy person...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0.936195,0,1
3,-1.990000e+18,From Lady Gaga's Bad Romance: /I want your psy...,"[4, 0, 0]",0.932406,0,1
4,2.910000e+18,The first woman was created from the rib of a ...,"[4, 1, 2, 0, 0, 0, 0, 0, 2]",0.976999,1,1


In [333]:
y_pred = rule['label']
y = rule['manual']

In [339]:
print('F1:', f1_score(y_pred, y, average= None))
cm = confusion_matrix(y_pred, y)
print('confusion Matrix:', cm)
print ('Recall:', recall_score(y_pred, y))
print ('Precision:', precision_score(y_pred, y))

F1: [0.98245614 0.89655172]
confusion Matrix: [[168   6]
 [  0  26]]
Recall: 1.0
Precision: 0.8125
